In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
chatbot_arena_conversations = pd.read_csv("lmsys-chatbot_arena_conversations-33k_clean.csv")
chatbot_arena_conversations.head()

,id,model_a,model_b,winner_model_a,winner_model_b,winner_tie,prompt,response_a,response_b
0,58210e39b3fd4441a2bd4a518bb44c2d,chatglm-6b,koala-13b,0,1,0,"[""What is the difference between OpenCL and CU...","[""OpenCL and CUDA are two different programmin...","[""OpenCL and CUDA are both programming languag..."
1,2564acd09e3942fd97657d05282d4389,oasst-pythia-12b,alpaca-13b,0,0,1,"[""Why did my parent not invite me to their wed...","[""It is possible that your parent did not invi...","[""It is likely that they wanted to keep the gu..."
2,90bfd142157948aba01931726c888e7f,koala-13b,oasst-pythia-12b,0,1,0,"[""Fuji vs. Nikon, which is better?""]","[""Both Fuji and Nikon are popular camera brand...","[""This is a subjective question and the answer..."
3,a7c5accc53e649a3bc6b2e41d962ebc4,vicuna-13b,oasst-pythia-12b,0,1,0,"[""How to build an arena for chatbots?""]","[""Building an arena for chatbots can be done b...","[""Building an arena for chatbots is a great wa..."
4,adf27e819a3c494cb6e993f0c660e097,vicuna-13b,koala-13b,1,0,0,"[""When is it today?""]","[""I'm sorry, I cannot determine the current da...","[""Today is February 23, 2023.""]"


In [3]:
original_train = pd.read_csv("../data/train.csv")
original_train.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [4]:
# for idx1, row1 in tqdm(chatbot_arena_conversations.iterrows(), total=len(chatbot_arena_conversations)):
#     for idx2, row2 in original_train.iterrows():
#         if row1["prompt"] == row2["prompt"]:
#             print(idx1, idx2)

In [5]:
prompt_sets = set()
response_a_sets = set()
response_b_sets = set()

duplicate_prompt_counts = 0
duplicate_response_a_counts = 0
duplicate_response_b_counts = 0

duplicate_prompt_index = []
duplicate_response_a_index = []
duplicate_response_b_index = []

## prompt
for idx1, row1 in tqdm(original_train.iterrows(), total=len(original_train)):
    if row1["prompt"] in prompt_sets:
        continue
    else:
        prompt_sets.add(row1["prompt"])

for idx2, row2 in tqdm(chatbot_arena_conversations.iterrows(), total=len(chatbot_arena_conversations)):
    if row2["prompt"] in prompt_sets:
        duplicate_prompt_counts += 1
        duplicate_prompt_index.append(idx2)

## response_a
for idx1, row1 in tqdm(original_train.iterrows(), total=len(original_train)):
    if row1["response_a"] in response_a_sets:
        continue
    else:
        response_a_sets.add(row1["response_a"])

for idx2, row2 in tqdm(chatbot_arena_conversations.iterrows(), total=len(chatbot_arena_conversations)):
    if row2["response_a"] in response_a_sets:
        duplicate_response_a_counts += 1
        duplicate_response_a_index.append(idx2)

## response_b
for idx1, row1 in tqdm(original_train.iterrows(), total=len(original_train)):
    if row1["response_b"] in response_b_sets:
        continue
    else:
        response_b_sets.add(row1["response_b"])

for idx2, row2 in tqdm(chatbot_arena_conversations.iterrows(), total=len(chatbot_arena_conversations)):
    if row2["response_b"] in response_b_sets:
        duplicate_response_b_counts += 1
        duplicate_response_b_index.append(idx2)

print(duplicate_prompt_counts)
print(duplicate_response_a_counts)
print(duplicate_response_b_counts)

duplicate_prompt_index_set = set(duplicate_prompt_index)
duplicate_response_a_index_set = set(duplicate_response_a_index)
duplicate_response_b_index_set = set(duplicate_response_b_index)

print("The intersection of duplicate_prompt_index and duplicate_response_a_index is: ", len(duplicate_prompt_index_set.intersection(duplicate_response_a_index_set)))
print("The intersection of duplicate_prompt_index and duplicate_response_b_index is: ", len(duplicate_prompt_index_set.intersection(duplicate_response_b_index_set)))
print("The intersection of duplicate_response_a_index and duplicate_response_b_index is: ", len(duplicate_response_a_index_set.intersection(duplicate_response_b_index_set)))

print("The intersection of duplicate_prompt_index and duplicate_response_a_index and duplicate_response_b_index is: ", len(duplicate_prompt_index_set.intersection(duplicate_response_a_index_set).intersection(duplicate_response_b_index_set)))

100%|██████████| 32970/32970 [00:01<00:00, 18791.31it/s]

8969
3361
3406
The intersection of duplicate_prompt_index and duplicate_response_a_index is:  2913
The intersection of duplicate_prompt_index and duplicate_response_b_index is:  2896
The intersection of duplicate_response_a_index and duplicate_response_b_index is:  2621
The intersection of duplicate_prompt_index and duplicate_response_a_index and duplicate_response_b_index is:  2465


In [6]:
chatbot_arena_conversations.drop(duplicate_prompt_index, inplace=True)
chatbot_arena_conversations.reset_index(drop=True, inplace=True)
print(chatbot_arena_conversations.shape)
chatbot_arena_conversations.to_csv("lmsys-chatbot_arena_conversations-24k.csv", index=False)

(24001, 9)


In [7]:
chatbot_arena_conversations = pd.read_csv("lmsys-chatbot_arena_conversations-33k_clean.csv")
chatbot_arena_conversations.head()

,id,model_a,model_b,winner_model_a,winner_model_b,winner_tie,prompt,response_a,response_b
0,58210e39b3fd4441a2bd4a518bb44c2d,chatglm-6b,koala-13b,0,1,0,"[""What is the difference between OpenCL and CU...","[""OpenCL and CUDA are two different programmin...","[""OpenCL and CUDA are both programming languag..."
1,2564acd09e3942fd97657d05282d4389,oasst-pythia-12b,alpaca-13b,0,0,1,"[""Why did my parent not invite me to their wed...","[""It is possible that your parent did not invi...","[""It is likely that they wanted to keep the gu..."
2,90bfd142157948aba01931726c888e7f,koala-13b,oasst-pythia-12b,0,1,0,"[""Fuji vs. Nikon, which is better?""]","[""Both Fuji and Nikon are popular camera brand...","[""This is a subjective question and the answer..."
3,a7c5accc53e649a3bc6b2e41d962ebc4,vicuna-13b,oasst-pythia-12b,0,1,0,"[""How to build an arena for chatbots?""]","[""Building an arena for chatbots can be done b...","[""Building an arena for chatbots is a great wa..."
4,adf27e819a3c494cb6e993f0c660e097,vicuna-13b,koala-13b,1,0,0,"[""When is it today?""]","[""I'm sorry, I cannot determine the current da...","[""Today is February 23, 2023.""]"


In [8]:
all_same_index = list(duplicate_prompt_index_set.intersection(duplicate_response_a_index_set).intersection(duplicate_response_b_index_set))
chatbot_arena_conversations.drop(all_same_index, inplace=True)
chatbot_arena_conversations.reset_index(drop=True, inplace=True)
print(chatbot_arena_conversations.shape)
chatbot_arena_conversations.to_csv("lmsys-chatbot_arena_conversations-30k.csv", index=False)

(30505, 9)
